# library

In [6]:
!pip install pyarabic
!pip install transformers
!pip install pytorch_lightning
!pip install torch
!pip install pad_sequences
!pip install pytorch_pretrained_bert
!pip install pytorch-pretrained-bert==0.4.0
!pip install seqeval==0.0.12
!pip install --upgrade urllib3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.11
    Uninstalling urllib3-1.26.11:
      Successfully uninstalled urllib3-1.26.11


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.26.11-py2.py3-none-any.whl (139 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packag

In [7]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import re
import string
import nltk 
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
import pyarabic.araby as araby
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc
import joblib
from transformers import AutoTokenizer, AutoModel
import torch
from torch import nn,optim
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report
from tqdm.auto import tqdm
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
from torchmetrics.functional import f1_score
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
from transformers import Trainer, TrainingArguments
import random



In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("your device ", device)

your device  cuda


In [9]:
print('Make sure cuda is installed:', torch.cuda.is_available())
print('Make sure cudnn is enabled:', torch.backends.cudnn.enabled)

Make sure cuda is installed: True
Make sure cudnn is enabled: True


In [10]:
Nile = pd.read_csv("/content/NileUniversity_Emotional (1).csv")
Nile.head(10)

,ID,text,label
0,1,الاوليمبياد الجايه هكون لسه ف الكليه ..,none
1,2,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,anger
2,3,كتنا نيله ف حظنا الهباب xD,sadness
3,4,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,joy
4,5,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,none
5,6,دلوقتي عندها اربع سنين .... ومحدش يقدر يكلمها ...,surprise
6,7,الحب يسلبها كيدها يرجعها الي براءتها الاولي وي...,love
7,8,she3er: شوفي بـ عيني امنيات الغلابه واحلام ثلث...,sympathy
8,9,86: صوتك هو حبيبي الثالث بعدك وبعد عيونك ~...,love
9,10,عمر عسر اتغلب وخرج وبيكمل مسيره مصر الفاشله في...,sadness


# PreProcessing

In [11]:
Nile.tail()

,ID,text,label
10060,10061,2222: يلا يا جماعه حفله عمرو دياب خلصت نريح شو...,sadness
10061,10062,Mohamed5: اييييه دااا 😲😲 اوزيييل❤,surprise
10062,10063,عملتلها ريتويت بمناسبه ساره بتاعه الاوليمبياد 😃,none
10063,10064,وعليك قبلنا يانجم النجوم ياعندليب الحب والاحساس,joy
10064,10065,AlHamad يطلع ننهم كل شي سيء ووضيع كل خساسه الع...,anger


In [12]:
Nile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10065 entries, 0 to 10064
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10065 non-null  int64 
 1   text    10064 non-null  object
 2   label   10065 non-null  object
dtypes: int64(1), object(2)
memory usage: 236.0+ KB


# Check nulls

In [13]:
Nile[Nile.text.isnull()]

,ID,text,label
8619,8620,NaN,love


In [14]:
Nile = Nile.dropna()

In [15]:
Nile.isnull().sum()

ID       0
text     0
label    0
dtype: int64

# remove Stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
stopwords_list = stopwords.words('arabic')
listToStr = ' '.join([str(elem) for elem in stopwords_list]) 
listToStr

'إذ إذا إذما إذن أف أقل أكثر ألا إلا التي الذي الذين اللاتي اللائي اللتان اللتيا اللتين اللذان اللذين اللواتي إلى إليك إليكم إليكما إليكن أم أما أما إما أن إن إنا أنا أنت أنتم أنتما أنتن إنما إنه أنى أنى آه آها أو أولاء أولئك أوه آي أي أيها إي أين أين أينما إيه بخ بس بعد بعض بك بكم بكم بكما بكن بل بلى بما بماذا بمن بنا به بها بهم بهما بهن بي بين بيد تلك تلكم تلكما ته تي تين تينك ثم ثمة حاشا حبذا حتى حيث حيثما حين خلا دون ذا ذات ذاك ذان ذانك ذلك ذلكم ذلكما ذلكن ذه ذو ذوا ذواتا ذواتي ذي ذين ذينك ريث سوف سوى شتان عدا عسى عل على عليك عليه عما عن عند غير فإذا فإن فلا فمن في فيم فيما فيه فيها قد كأن كأنما كأي كأين كذا كذلك كل كلا كلاهما كلتا كلما كليكما كليهما كم كم كما كي كيت كيف كيفما لا لاسيما لدى لست لستم لستما لستن لسن لسنا لعل لك لكم لكما لكن لكنما لكي لكيلا لم لما لن لنا له لها لهم لهما لهن لو لولا لوما لي لئن ليت ليس ليسا ليست ليستا ليسوا ما ماذا متى مذ مع مما ممن من منه منها منذ مه مهما نحن نحو نعم ها هاتان هاته هاتي هاتين هاك هاهنا هذا هذان هذه هذي هذين هكذا هل هلا هم هما هن هنا هن

In [18]:
print(len(stopwords_list))


754


# Clean text

In [19]:
for letter in '#.][!XR':
    Nile['text'] = Nile['text'].astype(str).str.replace(letter,'')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


# punctuation

In [20]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

# Normalize

In [21]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [22]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [23]:
def remove(string):
    return " ".join(string.split())

In [24]:
Nile.tail()

,ID,text,label
10060,10061,2222: يلا يا جماعه حفله عمرو دياب خلصت نريح شو...,sadness
10061,10062,Mohamed5: اييييه دااا 😲😲 اوزيييل❤,surprise
10062,10063,عملتلها ريتويت بمناسبه ساره بتاعه الاوليمبياد 😃,none
10063,10064,وعليك قبلنا يانجم النجوم ياعندليب الحب والاحساس,joy
10064,10065,AlHamad يطلع ننهم كل شي سيء ووضيع كل خساسه الع...,anger


# Remove prefixes and suffixes

In [25]:
st = ISRIStemmer()
def roots(text):
    return st.stem(text)

In [26]:
def pre(text):
    return st.pre32(text)

In [27]:
def suf(text):
    return st.suf32(text)

In [28]:
def processPost(text): 
    # remove Negation
    text=text.replace("ﷺ", "")
    text=text.replace("لا", "")
    text=text.replace("ما", "")
    text=text.replace("لم", "")
    text=text.replace("بدون", "")
    text=text.replace("من دون", "")
    text=text.replace("لن", "")
    text=text.replace("لات", "")
   
    #Replace @username with empty string
    text = re.sub('@[^\s]+', ' ', text)
    # remove english letters
    text = re.sub('[a-zA-Z0-9_]|#|http\S+', '', text)
    
    #remove links
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    
    #Replace #word with word
    text = re.sub(r'#([^\s]+)', r'\1', text)

    # remove punctuations
    text = remove_punctuations(text)
    
    # normalize the tweet
    text = normalize_arabic(text)
    # remove Diacritics
    text = araby.strip_diacritics(text)
    
    # remove repeated letters
    text = remove_repeating_char(text)
    # remove space
    text = remove(text)
    text = ' '.join(word for word in text.split(' ') if word not in stopwords_list)
    #remove uni code 
    #text = re.sub(r'\W+', ' ', text)
    # remove suffixes
    text= suf(text)
    # remove prefixes
    text= pre(text)
    
    return text

In [29]:
Nile["text"] = Nile['text'].apply(lambda x: processPost(x))

In [30]:
Nile['text']

0                         اوليمبياد الجايه هكون لسه الكليه
1        عجز اوازنه وصل اتج احلي يعني لسه اقل نفلس بهاي...
2                                    كتنا نيله حظنا الهباب
3          جميعنا نريد تحقيق اهدافنا تونس تالقت حراسه ارمي
4        اوليمبياد نظامها مختلف ومواعيد اونديال مكانتش ...
                               ...                        
10060    جعه حفله عمرو دياب خلصت نريح شويه ونبدا نكتئب ...
10061                                      ايه دا 😲 اوزيل❤
10062        عملتلها ريتويت بمناسبه ساره بتاعه اوليمبياد 😃
10063            وعليك قبا يانجم اجوم ياعندليب الحب واحساس
10064    يطلع نهم شي سيء وضيع خساسه العا تجمعت ايرانين ...
Name: text, Length: 10064, dtype: object

In [31]:
Nile.tail()

,ID,text,label
10060,10061,جعه حفله عمرو دياب خلصت نريح شويه ونبدا نكتئب ...,sadness
10061,10062,ايه دا 😲 اوزيل❤,surprise
10062,10063,عملتلها ريتويت بمناسبه ساره بتاعه اوليمبياد 😃,none
10063,10064,وعليك قبا يانجم اجوم ياعندليب الحب واحساس,joy
10064,10065,يطلع نهم شي سيء وضيع خساسه العا تجمعت ايرانين ...,anger


# Save cleaning_data

In [32]:
Nile.to_csv('clean_data.csv',index=False)

# Reducing words to their roots

In [33]:
Nile['text'] = Nile['text']. apply(lambda x: [st.stem(y) for y in x.split()])
Nile['text'] = Nile['text'].apply(lambda x: ' '.join([word for word in x ]))
Nile['text']

0                                اوليمبياد جيه هكن لسه كله
1        عجز وزن وصل اتج احل يعن لسه اقل فلس هيم لسه تب...
2                                          كتن نيل حظن هبب
3                          جمع نرد حقق هدف ونس تلق حرس ارم
4        اوليمبياد نظم خلف مواعيد ديل مكانتش قرف حجه كا...
                               ...                        
10060    جعه حفل عمرو ديب خلص نرح شوه نبد كتئب عشم نفس ...
10061                                        ايه دا 😲 زيل❤
10062                   عملتل ريتو نسب سره بتع اوليمبياد 😃
10063                     وعل قبا نجم اجم ياعندليب لحب حسس
10064    طلع نهم شي سيء وضع خسس الع جمع يرن الي لفت بحر...
Name: text, Length: 10064, dtype: object

In [34]:
Nile.to_csv('clean_data_2.csv',index=False)

# data with_out ISRIStemmer

In [35]:
path_input = './clean_data.csv'
df = pd.read_csv(path_input)
df.head()

,ID,text,label
0,1,اوليمبياد الجايه هكون لسه الكليه,none
1,2,عجز اوازنه وصل اتج احلي يعني لسه اقل نفلس بهاي...,anger
2,3,كتنا نيله حظنا الهباب,sadness
3,4,جميعنا نريد تحقيق اهدافنا تونس تالقت حراسه ارمي,joy
4,5,اوليمبياد نظامها مختلف ومواعيد اونديال مكانتش ...,none


In [36]:
df.dropna(inplace=True)
df.isnull().sum()

ID       0
text     0
label    0
dtype: int64

# data with ISRIStemmer

In [37]:
path_input = './clean_data_2.csv'
dff = pd.read_csv(path_input)
dff.head()

,ID,text,label
0,1,اوليمبياد جيه هكن لسه كله,none
1,2,عجز وزن وصل اتج احل يعن لسه اقل فلس هيم لسه تب...,anger
2,3,كتن نيل حظن هبب,sadness
3,4,جمع نرد حقق هدف ونس تلق حرس ارم,joy
4,5,اوليمبياد نظم خلف مواعيد ديل مكانتش قرف حجه كا...,none


In [38]:
dff.dropna(inplace=True)
dff.isnull().sum()

ID       0
text     0
label    0
dtype: int64

# Model without ISRIStemmer

In [39]:
train, val = train_test_split(df[['label','text']], test_size=0.1, random_state=333)


lbl_enc = LabelEncoder()
train.loc[:,"label"] = lbl_enc.fit_transform(train["label"])
val.loc[:,"label"] = lbl_enc.transform(val["label"])

joblib.dump(lbl_enc,"label_encoder.pkl")
train.to_csv("train.csv",index=False)
val.to_csv("val.csv",index=False)

In [40]:
train

,label,text
1626,4,وش هالجو الساعه تابع اوليمبياد
8559,7,صور مؤه احم بيوت 💔 الهم اعنهم وفرج كربتهم وانع...
9019,7,بصراحه متعاطف شعب امريكي ت
8200,6,انتوا ازاي قدرتوا تحدوا اسل البلد اهم العميد د...
8648,7,ه يستر عليكم ويشيل هاحنه واله قلوبنا معكمصر تس...
...,...,...
7538,6,انا مش فاهمه 😂
6070,0,انت خورم كبير ياشريف ياريت تمشي وتريحنا وخد اح...
9643,4,تخيل الطاوله الدومنه دخلو ضمن مسابقات اوليمبيا...
973,0,غالي متطلعش دين ابو


In [41]:
val

,label,text
3175,3,زوجين يحبون انجبوا شوق هيام
7156,7,سم اله قلبي عليكم ياهل بيت ❤️ حبيبي حسين زياره
3054,1,شنط السفر مفيهاش مكان مليانه خوف اصير
362,6,اشهر مشهد بلحه
9581,7,مؤثر جدا اتصال موظفه شركه اتصات البلجيكيه بامر...
...,...,...
7052,4,فع موقوفين الفيفا الجنه اوليمبيه بسب التدخل ال...
5341,3,افريقيا الف مليون تحيه لكابتن مؤمن سلين وش الس...
5966,1,وانا ايه الي هيسليني اوليمبياد تخلص
9388,0,صادق اطيمش اسم السياسي عاه اجتمعات اسميه


In [42]:
lbl_enc.classes_
{v: k for v, k in enumerate(lbl_enc.classes_)}

{0: 'anger',
 1: 'fear',
 2: 'joy',
 3: 'love',
 4: 'none',
 5: 'sadness',
 6: 'surprise',
 7: 'sympathy'}

In [43]:
class ArabicDataset(Dataset):
    def __init__(self,data,max_len,model_type="Mini"):
        super().__init__()
        self.labels = data["label"].values
        self.texts = data["text"].values
        self.max_len = max_len
        model = {"Twitter": "aubmindlab/bert-base-arabertv02-twitter",
                "Base": "aubmindlab/bert-base-arabertv02",
                 "MAR" : "UBC-NLP/MARBERT"}
        self.tokenizer = AutoTokenizer.from_pretrained(model[model_type])
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self,idx):
        text = " ".join(self.texts[idx].split())
        label = self.labels[idx]
        inputs = self.tokenizer(text,padding='max_length',
                                max_length=self.max_len,truncation=True,return_tensors="pt")
        #input_ids,token_type_ids,attention_mask
        return {
            "inputs":{"input_ids":inputs["input_ids"][0],
                      "token_type_ids":inputs["token_type_ids"][0],
                      "attention_mask":inputs["attention_mask"][0],
                     },
            "labels": torch.tensor(label,dtype=torch.long) 
        }

In [44]:
class ArabicDataModule(pl.LightningDataModule):
    def __init__(self,train_path,val_path,batch_size=12,max_len=100,model_type="Twitter"):
        super().__init__()
        self.train_path,self.val_path= train_path,val_path
        self.batch_size = batch_size
        self.max_len = max_len
        self.model_type = model_type
    
    def setup(self,stage=None):
        train = pd.read_csv(self.train_path)
        val = pd.read_csv(self.val_path)
        self.train_dataset = ArabicDataset(data=train,max_len=self.max_len,model_type=self.model_type)
        self.val_dataset = ArabicDataset(data=val,max_len=self.max_len,model_type=self.model_type)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset,batch_size=self.batch_size,shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False)
    
    def test_dataloader(self):
        return DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False)

In [45]:
n_classes = 8
class ArabicBertModel(pl.LightningModule):
    def __init__(self,model_type="Mini"):
        super().__init__()
        model = {"Twitter": ("aubmindlab/bert-base-arabertv02-twitter",768),
                 "Base": ("aubmindlab/bert-base-arabertv02",768),
                 "MAR" : ("UBC-NLP/MARBERT",768)}
        self.bert_model = AutoModel.from_pretrained(model[model_type][0])
        self.fc = nn.Linear(model[model_type][1],n_classes)
    
    def forward(self,inputs):
        out = self.bert_model(**inputs)#inputs["input_ids"],inputs["token_type_ids"],inputs["attention_mask"])
        last_hidden_states = out[1]
        out = self.fc(last_hidden_states)
        return out
    
    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=0.0001)
    
    def criterion(self,output,target):
        return nn.CrossEntropyLoss()(output,target)
    
    #TODO: adding metrics
    def training_step(self,batch,batch_idx):
        x,y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_Score = f1_score(out, y, num_classes=n_classes, average='macro')
        metrics = {"train_f1": f1_Score, "train_loss": loss}
        self.log_dict(metrics)
        return loss
    
    def validation_step(self,batch,batch_idx):
        x, y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_Score = f1_score(out, y, num_classes=n_classes, average='macro')
        metrics = {"val_f1": f1_Score, "val_loss": loss}
        self.log_dict(metrics)
        return metrics

# Arabert Twitter model

In [46]:
# TODO: getting different models sizes results
MODEL_TYPE = "Twitter"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02-twitter were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dens

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type      | Params
-----------------------------------------
0 | bert_model | BertModel | 135 M 
1 | fc         | Linear    | 6.2 K 
-----------------------------------------
135 M     Trainable params
0         Non-trainable params
135 M     Total params
540.798   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))

100%|██████████| 11/11 [00:04<00:00,  2.43it/s]

              precision    recall  f1-score   support

       anger       0.75      0.75      0.75       130
        fear       0.99      0.95      0.97       128
         joy       0.66      0.63      0.64       124
        love       0.73      0.85      0.78       110
        none       0.72      0.73      0.72       169
     sadness       0.69      0.54      0.60       137
    surprise       0.51      0.65      0.57        93
    sympathy       0.93      0.90      0.91       115

    accuracy                           0.75      1006
   macro avg       0.75      0.75      0.74      1006
weighted avg       0.75      0.75      0.75      1006



# Arabert Base Model

In [48]:
# TODO: getting different models sizes results
MODEL_TYPE = "Base"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: Lig

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type      | Params
-----------------------------------------
0 | bert_model | BertModel | 135 M 
1 | fc         | Linear    | 6.2 K 
-----------------------------------------
135 M     Trainable params
0         Non-trainable params
135 M     Total params
540.798   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))


100%|██████████| 11/11 [08:20<00:00, 45.51s/it]

100%|██████████| 11/11 [00:04<00:00,  2.42it/s]

              precision    recall  f1-score   support

       anger       0.70      0.78      0.73       130
        fear       0.91      0.94      0.92       128
         joy       0.76      0.38      0.51       124
        love       0.65      0.90      0.76       110
        none       0.74      0.68      0.71       169
     sadness       0.59      0.63      0.61       137
    surprise       0.45      0.53      0.49        93
    sympathy       0.93      0.85      0.89       115

    accuracy                           0.71      1006
   macro avg       0.72      0.71      0.70      1006
weighted avg       0.72      0.71      0.71      1006



# MARBERT

In [50]:
# TODO: getting different models sizes results
MODEL_TYPE = "MAR"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type      | Params
-----------------------------------------
0 | bert_model | BertModel | 162 M 
1 | fc         | Linear    | 6.2 K 
-----------------------------------------
162 M     Trainable params
0         Non-trainable params
162 M     Total params
651.390   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [51]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))

100%|██████████| 11/11 [00:04<00:00,  2.42it/s]

              precision    recall  f1-score   support

       anger       0.69      0.66      0.67       130
        fear       0.99      0.94      0.96       128
         joy       0.58      0.55      0.56       124
        love       0.72      0.81      0.76       110
        none       0.68      0.73      0.70       169
     sadness       0.62      0.54      0.58       137
    surprise       0.44      0.53      0.48        93
    sympathy       0.92      0.84      0.88       115

    accuracy                           0.70      1006
   macro avg       0.70      0.70      0.70      1006
weighted avg       0.71      0.70      0.70      1006



# with ISRIStemmer

In [52]:
Nile['text']

0                                اوليمبياد جيه هكن لسه كله
1        عجز وزن وصل اتج احل يعن لسه اقل فلس هيم لسه تب...
2                                          كتن نيل حظن هبب
3                          جمع نرد حقق هدف ونس تلق حرس ارم
4        اوليمبياد نظم خلف مواعيد ديل مكانتش قرف حجه كا...
                               ...                        
10060    جعه حفل عمرو ديب خلص نرح شوه نبد كتئب عشم نفس ...
10061                                        ايه دا 😲 زيل❤
10062                   عملتل ريتو نسب سره بتع اوليمبياد 😃
10063                     وعل قبا نجم اجم ياعندليب لحب حسس
10064    طلع نهم شي سيء وضع خسس الع جمع يرن الي لفت بحر...
Name: text, Length: 10064, dtype: object

In [53]:
train, val = train_test_split(dff[['label','text']], test_size=0.1, random_state=333)


lbl_enc = LabelEncoder()
train.loc[:,"label"] = lbl_enc.fit_transform(train["label"])
val.loc[:,"label"] = lbl_enc.transform(val["label"])

joblib.dump(lbl_enc,"label_encoder.pkl")
train.to_csv("train.csv",index=False)
val.to_csv("val.csv",index=False)

In [54]:
lbl_enc.classes_
{v: k for v, k in enumerate(lbl_enc.classes_)}

{0: 'anger',
 1: 'fear',
 2: 'joy',
 3: 'love',
 4: 'none',
 5: 'sadness',
 6: 'surprise',
 7: 'sympathy'}

In [55]:
train

,label,text
1626,4,وش هالجو سعه تبع اوليمبياد
8559,7,صور مؤه احم بيت 💔 لهم اعن فرج كرب نعم علي
9019,7,صرح عاطف شعب امر ت
8200,6,انت ازي قدر تحد اسل بلد اهم عمد دا رد يقل احن ...
8648,7,ه ستر علي يشل هحن وله قلب معكمصر تهل
...,...,...
7538,6,انا مش فهم 😂
6070,0,انت خورم كبر شرف يري تمش ترح وخد حمد عدل عاك ع...
9643,4,تخل طول دوم دخلو ضمن سبق اوليمبياد كنا خدن ذهب...
973,0,غلي طلعش دين ابو


In [56]:
class ArabicDataset(Dataset):
    def __init__(self,data,max_len,model_type="Mini"):
        super().__init__()
        self.labels = data["label"].values
        self.texts = data["text"].values
        self.max_len = max_len
        model = {"Twitter": "aubmindlab/bert-base-arabertv02-twitter",
                "Base": "aubmindlab/bert-base-arabertv02",
                 "MAR": "UBC-NLP/MARBERT"}
        self.tokenizer = AutoTokenizer.from_pretrained(model[model_type])
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self,idx):
        text = " ".join(self.texts[idx].split())
        label = self.labels[idx]
        inputs = self.tokenizer(text,padding='max_length',
                                max_length=self.max_len,truncation=True,return_tensors="pt")
        #input_ids,token_type_ids,attention_mask
        return {
            "inputs":{"input_ids":inputs["input_ids"][0],
                      "token_type_ids":inputs["token_type_ids"][0],
                      "attention_mask":inputs["attention_mask"][0],
                     },
            "labels": torch.tensor(label,dtype=torch.long) 
        }

In [57]:
class ArabicDataModule(pl.LightningDataModule):
    def __init__(self,train_path,val_path,batch_size=12,max_len=100,model_type="Twitter"):
        super().__init__()
        self.train_path,self.val_path= train_path,val_path
        self.batch_size = batch_size
        self.max_len = max_len
        self.model_type = model_type
    
    def setup(self,stage=None):
        train = pd.read_csv(self.train_path)
        val = pd.read_csv(self.val_path)
        self.train_dataset = ArabicDataset(data=train,max_len=self.max_len,model_type=self.model_type)
        self.val_dataset = ArabicDataset(data=val,max_len=self.max_len,model_type=self.model_type)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset,batch_size=self.batch_size,shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False)
    
    def test_dataloader(self):
        return DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False)

In [58]:
n_classes = 8
class ArabicBertModel(pl.LightningModule):
    def __init__(self,model_type="Mini"):
        super().__init__()
        model = {"Twitter": ("aubmindlab/bert-base-arabertv02-twitter",768),
                 "Base": ("aubmindlab/bert-base-arabertv02",768),
                 "MAR": ("UBC-NLP/MARBERT",768)}
        self.bert_model = AutoModel.from_pretrained(model[model_type][0])
        self.fc = nn.Linear(model[model_type][1],n_classes)
    
    def forward(self,inputs):
        out = self.bert_model(**inputs)#inputs["input_ids"],inputs["token_type_ids"],inputs["attention_mask"])
        last_hidden_states = out[1]
        out = self.fc(last_hidden_states)
        return out
    
    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=0.0001)
    
    def criterion(self,output,target):
        return nn.CrossEntropyLoss()(output,target)
    
    #TODO: adding metrics
    def training_step(self,batch,batch_idx):
        x,y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_Score = f1_score(out, y, num_classes=n_classes, average='macro')
        metrics = {"train_f1": f1_Score, "train_loss": loss}
        self.log_dict(metrics)
        return loss
    
    def validation_step(self,batch,batch_idx):
        x, y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_Score = f1_score(out, y, num_classes=n_classes, average='macro')
        metrics = {"val_f1": f1_Score, "val_loss": loss}
        self.log_dict(metrics)
        return metrics

# Arabert twitter model

In [59]:
# TODO: getting different models sizes results
MODEL_TYPE = "Twitter"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02-twitter were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dens

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [60]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))


100%|██████████| 11/11 [12:00<00:00, 65.48s/it]

100%|██████████| 11/11 [00:04<00:00,  2.42it/s]

              precision    recall  f1-score   support

       anger       0.68      0.65      0.66       130
        fear       0.97      0.92      0.94       128
         joy       0.54      0.52      0.53       124
        love       0.77      0.75      0.76       110
        none       0.68      0.77      0.72       169
     sadness       0.57      0.65      0.61       137
    surprise       0.52      0.44      0.48        93
    sympathy       0.87      0.82      0.84       115

    accuracy                           0.70      1006
   macro avg       0.70      0.69      0.69      1006
weighted avg       0.70      0.70      0.70      1006



# Arabert Base model

In [61]:
# TODO: getting different models sizes results
MODEL_TYPE = "Base"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: Lig

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [62]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))

100%|██████████| 11/11 [00:04<00:00,  2.41it/s]

              precision    recall  f1-score   support

       anger       0.73      0.58      0.64       130
        fear       0.94      0.91      0.93       128
         joy       0.53      0.50      0.51       124
        love       0.77      0.66      0.71       110
        none       0.65      0.76      0.70       169
     sadness       0.51      0.62      0.56       137
    surprise       0.54      0.41      0.47        93
    sympathy       0.78      0.90      0.83       115

    accuracy                           0.68      1006
   macro avg       0.68      0.67      0.67      1006
weighted avg       0.68      0.68      0.68      1006



# MARBERT

In [63]:
# TODO: getting different models sizes results
MODEL_TYPE = "MAR"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [64]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))


100%|██████████| 11/11 [08:21<00:00, 45.63s/it]

100%|██████████| 11/11 [00:04<00:00,  2.44it/s]

              precision    recall  f1-score   support

       anger       0.71      0.53      0.61       130
        fear       0.98      0.91      0.95       128
         joy       0.51      0.38      0.44       124
        love       0.70      0.79      0.74       110
        none       0.57      0.98      0.72       169
     sadness       0.56      0.47      0.51       137
    surprise       0.63      0.33      0.44        93
    sympathy       0.82      0.84      0.83       115

    accuracy                           0.67      1006
   macro avg       0.68      0.66      0.65      1006
weighted avg       0.68      0.67      0.66      1006



# TFIDF

In [65]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 1),
    max_features =10000)

unigramdataGet= word_vectorizer.fit_transform(dff['text'].astype('str'))
unigramdataGet = unigramdataGet.toarray()

vocab = word_vectorizer.get_feature_names()
unigramdata_features=pd.DataFrame(np.round(unigramdataGet, 1), columns=vocab)
unigramdata_features[unigramdata_features>0] = 1

unigramdata_features.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,0في,0وك,TM,ıs,ɑɺi,ρɺɹ,алепо,детеи,из,мстит,...,ڪم,ڪيب,ګبر,ګثر,ھم,ۆايام,ۆرق,ۆفا,یفق,۶ظم
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
pro= preprocessing.LabelEncoder()
encpro=pro.fit_transform(dff['label'])
dff['label'] = encpro
dff['label']

0        4
1        0
2        5
3        2
4        4
        ..
10059    5
10060    6
10061    4
10062    2
10063    0
Name: label, Length: 10056, dtype: int64

In [67]:
y=dff['label']
X=unigramdata_features

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=333)

# LogisticRegression 

In [69]:
LR= LogisticRegression(penalty = 'l2', C = 1)
LR= LR.fit(X_train , y_train)
y_pred = LR.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[152   1  23   5  13  39  24   6]
 [  1 227   2   0   6   2   5   1]
 [ 18   0 119  31  47  16  19   2]
 [ 13   1  27 180   0  19   5   6]
 [  9   0  20   0 264  18  13   2]
 [ 35   2  19  21  35 101  32   6]
 [ 18   0  26   4  47  20  95   0]
 [  1   1   5  16   0   8   2 182]]


              precision    recall  f1-score   support

           0       0.62      0.58      0.60       263
           1       0.98      0.93      0.95       244
           2       0.49      0.47      0.48       252
           3       0.70      0.72      0.71       251
           4       0.64      0.81      0.72       326
           5       0.45      0.40      0.43       251
           6       0.49      0.45      0.47       210
           7       0.89      0.85      0.87       215

    accuracy                           0.66      2012
   macro avg       0.66      0.65      0.65      2012
weighted avg       0.65      0.66      0.65      2012



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


# SVM

In [70]:
from sklearn.svm import SVC
clf=SVC(kernel='linear')
clf.fit(X_train , y_train)
y_pred =clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[150   5  29  11  13  28  21   6]
 [  1 228   1   0   6   5   2   1]
 [ 23   0 130  25  40  12  17   5]
 [ 15   3  37 170   0  15   6   5]
 [  9   1  26   1 256  16  15   2]
 [ 29   6  30  19  39  91  29   8]
 [ 24   0  27   2  44  23  89   1]
 [  4   3   7  16   0   7   3 175]]


              precision    recall  f1-score   support

           0       0.59      0.57      0.58       263
           1       0.93      0.93      0.93       244
           2       0.45      0.52      0.48       252
           3       0.70      0.68      0.69       251
           4       0.64      0.79      0.71       326
           5       0.46      0.36      0.41       251
           6       0.49      0.42      0.45       210
           7       0.86      0.81      0.84       215

    accuracy                           0.64      2012
   macro avg       0.64      0.64      0.64      2012
weighted avg       0.64      0.64      0.64      2012



# OTP

In [71]:
X = np.array(X_train)
Y = np.array( y_train)
# Always scale the input. The most convenient way is to use a pipeline.
clf = make_pipeline(StandardScaler(),
                    SGDClassifier(max_iter=1000, tol=1e-3))
clf.fit(X, Y)


y_pred =clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


[[157   0  31   4  14  35  17   5]
 [  6 196  10   7   7  10   7   1]
 [ 17   1 135  19  43  20  14   3]
 [ 13   1  45 152   0  32   4   4]
 [ 15   0  38   1 230  23  17   2]
 [ 27   7  39  11  34 111  19   3]
 [ 17   0  42   4  48  28  71   0]
 [ 12   2  25   8   0  17   2 149]]


              precision    recall  f1-score   support

           0       0.59      0.60      0.60       263
           1       0.95      0.80      0.87       244
           2       0.37      0.54      0.44       252
           3       0.74      0.61      0.67       251
           4       0.61      0.71      0.66       326
           5       0.40      0.44      0.42       251
           6       0.47      0.34      0.39       210
           7       0.89      0.69      0.78       215

    accuracy                           0.60      2012
   macro avg       0.63      0.59      0.60      2012
weighted avg       0.62      0.60      0.60      2012

